In [216]:
import numpy as np

import time

from scipy.sparse import diags
from scipy.sparse import eye
from scipy.sparse.linalg import spsolve

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row, grid

output_notebook(hide_banner=True)

# Résolution numérique d'EDP paraboliques

## Equation de la chaleur

Soient $\nu>0$ un coefficient de diffusion, un intervalle $\Omega=]0,L[$, $L>0$, et $u^0\in C^0(\Omega)$ une donnée initiale. On considère l'équation de la chaleur :

\begin{equation*}
\left\{
\begin{aligned}
  &\displaystyle\frac{\partial u}{\partial t}(t,x)-\nu\frac{\partial^2 u}{\partial x^2}(t,x) = 0, \quad &\forall~(t,x) \in\mathbb{R}_+^*\times\Omega, \\
  &\frac{\partial u}{\partial x}(t,0) = 0 = \frac{\partial u}{\partial x}(t,L )\quad &\forall~t\in\mathbb{R}_+^*, \\
  &u(x,0) = u^0(x), \quad &\forall x\in\Omega.
\end{aligned}
\right.
\end{equation*}
  

Pour la discrétisation en temps, on fixe un temps final $T>0$, puis on considère :
\begin{equation*}
    N\in\mathbb{R}^*, \qquad \Delta t = \frac{T}{N} \quad \text{et} \quad t^n = n\Delta t,\ n = 0, \dots, N.
\end{equation*}


Pour la discrétisation en espace, on considère :
\begin{equation*} J\in\mathbb{N}^*, \qquad \Delta x = \frac{L}{J+1} \quad\text{et}\quad x_j = j\Delta x,\ j = -1, \dots, J+2. \end{equation*}

On note $U^0_j = u^0(x_j)\ j = 0, \dots, J+1$, et on va chercher à construire de manière itérative une suite 
$U^n = \left(U^n_j\right)_{0\leq j \leq J+1}$ telle que $U^n_j$ approche $u(t^n,x_j)$.

Pour les questions d'implémentation, on prendra
\begin{equation*}
\nu = 1,\quad L=10,\quad J=1000, \quad u^0(x) = 2 - \cos \frac{2\pi x}{L},
\end{equation*}

In [217]:
# Commencons par définir les constantes et u0 qui nous serviront dans tout l'exercice
nu = 1
L = 10
J = 100
def u0(x) :
    return 2-np.cos(2*np.pi*x/L)

### Schéma explicite en temps

On considère le schéma "différences finies" explicite en temps donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^n_{j-1}-2U^n_j+U^n_{j+1}}{(\Delta x)^2} = 0 \qquad \forall~n\geq 0,\quad \forall~0 \leq j\leq J+1.
\end{equation*}

Afin d'approcher les conditions aux bords, on pourra utiliser les conventions :

\begin{equation}
  \label{Neumann1} 
  \frac{U_0^n - U^n_{-1}}{\Delta x} = 0 \quad\text{et}\quad \frac{U^n_{J+2} - U^n_{J+1}}{\Delta x} = 0, 
\end{equation}

ou

\begin{equation}
  \label{Neumann2} 
  \frac{U_1^n - U^n_{-1}}{2\Delta x} = 0 \quad\text{et}\quad \frac{U^n_{J+2} - U^n_{J}}{2\Delta x} = 0. 
\end{equation}



**Implémenter le schéma "différences finies" explicite en temps pour les deux conventions  $(1)$ et $(2)$.**  

**Commentaire :**  
Les matrices que nous allons utiliser sont tridiagonales, nous allons à chaque fois faire plusieurs multiplications de ces matrices par des vecteurs, et elles contiennent trop des valeurs non utilisées (zéros), c'est pourquoi je vais utiliser le stockage tridiagonal présenté dans la PC6. Je definis ci dessous la fonction *multiplier* qui multiplie une matrice tridiagonale par un vecteur.  
En termes de complexité, la multiplication normale d'une matrice tridiagonale de taille JxJ par un vecteur de taille J est de complexité $O(J^2)$, mais avec les matrices diagonales elle devient de complexité $O(J)$

In [218]:
def norme2(U) :
    return np.sqrt(np.sum(U*U))

def masse(U) :
    return np.sum(U)

# multiplier() prend en paramètre une matrice tridiagonale A (stockage tridiagonal) et un vecteur U
# et elle retourne le vecteur A.U
def multiplier(A, U) :
    p = np.zeros(len(U))
    p[0] = A[1][0]*U[0] + A[0][1]*U[1]
    p[-1] = A[1][-1]*U[-1] + A[2][-2]*U[-2]
    for i in range(1,len(U)-1) :
        p[i] = A[2][i-1]*U[i-1] + A[1][i]*U[i] + A[0][i+1]*U[i+1]
    return p

In [219]:
# La fonction resoudre_expl() prend en argument une matrice tridiagonale, un entier N (taille de t), un vecteur initial U0,
# elle renvoit le vecteur U^(N) calculé par U^(n+1) = A*U^(n), et les vecteurs de masse et d'energie correspondant à ce calcul
def resoudre_expl(A, N, U) :
    M = np.zeros(N+1) #masse
    H = np.zeros(N+1) #energie
    M[0] = masse(U)
    H[0] = dx*norme2(U)
    for n in range(1,N+1) :
        U = multiplier(A,U)
        M[n] = masse(U)
        H[n] = dx*norme2(U)
    return U,M,H

**Pour chaque traitement des conditions aux bords $(1)$ et $(2)$, utiliser $\Delta t= 0.004$ et afficher les solutions obtenues pour $T=0$, $T=1$, $T=5$ et $T=20$  (sur un même graphique, avec différentes couleurs). Afficher également la masse $\sum_{j=0}^{J+1} U^{n+1}_j$ en fonction de $n$, et l'énergie $\left\Vert U^n \right\Vert_{L^2}$ en fonction de $n$. Répéter ces expériences avec $\Delta t= 0.005$ et $T=0$, $T=1$, $T=2$ et $T=5$. Calculer $\alpha=\frac{\nu \Delta t}{\Delta x^2}$ dans les deux cas, et commenter les différents résultats obtenus.**

In [220]:
# La matrice de la méthode explicite pour la 1ere convention au bords : -----------------------------------------------
def A1_expl(alpha) :
    A1 = np.zeros((3,J+2))
    for i in range(J+1) :
        A1[0][i+1] = A1[2][i] = alpha
        A1[1][i] = 1-2*alpha
    A1[1][0] = A1[1][-1] = 1-alpha
    return A1

# La matrice de la méthode explicite pour la 2eme convention au bords : -----------------------------------------------
def A2_expl(alpha) :
    A2 = np.zeros((3,J+2))
    for i in range(J+1) :
        A2[0][i+1] = A2[2][i] = alpha
        A2[1][i] = 1-2*alpha
    A2[1][-1] = 1-2*alpha
    A2[0][1] = A2[2][-2] = 2*alpha
    return A2

**Commentaire :**  
Notons pour $x\in \Omega$ : $u_f(x) = \lim_{t=+\infty} u(x,t)$.
Théoriquement, la fonction $u_f$ est constante. Pour calculer sa valeur, il suffit d'utiliser la conservation de la masse. On trouve :
$$ \forall x \in \Omega, \quad L.u_f(x) = \int_{\Omega} u^0(x)dx = 2L $$
donc $u_f = 2$. Nous allons calculer dans la suite l'erreur finale en norme 2 entre $u_f$ et la solution calculée par chacun des deux shémas.

In [221]:
# CHOSIR ICI LA VALEUR DE dt ET 3 VALEURS DE T NON NULLES (la courbe pour T=0 sera toujours tracée)
dt = 0.004
#dt = 0.005
valeursT = [1,5,20]
#----------------------------------------------------------------------------------------------
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)

# Résolution et figure : ----------------------------------------------------------------------
# Calcul des matrices A1 et A2
A1 = A1_expl(alpha)
A2 = A2_expl(alpha)

col = {valeursT[0]:"green", valeursT[1]:"orange",valeursT[2]:"red" }

# 1er shéma
fig1 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 1 (méthode explicite)")
fig1.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T dans {1,5,20}, on calcule le vecteur U correspondant à l'instant T
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U1, M1, H1 = resoudre_expl(A1,N,U0)
    fig1.line(x, U1, color = col[T], legend = "T = "+str(T))

# 2ème shéma
fig2 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 2 (méthode explicite)")
fig2.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T, on calcule le vecteur U correspondant à l'instant T, et on le trace
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U2, M2, H2 = resoudre_expl(A2,N,U0)
    fig2.line(x, U2, color = col[T], legend = "T = "+str(T))

# évolution de la masse et de l'energie en fonction du temps pour les deux shémas
N = int(valeursT[-1]/dt)
t = np.array([n*dt for n in range(N+1)])

figM = figure(width = 490, height = 300, title = "Evolution de la masse en fonction du temps")
figM.line(t, M1, color = "red", legend = "pour le shéma 1")
figM.line(t, M2, legend = "pour le shéma 2")

figH = figure(width = 490, height = 300, title = "Evolution de l'energie en fonction du temps")
figH.line(t, H1, color = "red", legend = "pour le shéma 1")
figH.line(t, H2, legend = "pour le shéma 2")

print("à l'instant T = 20 :")
print("L'erreur en norme 2 entre la solution calculée avec le shéma 1 et uf est : ", norme2(U1-2))
print("L'erreur en norme 2 entre la solution calculée avec le shéma 2 et uf est : ", norme2(U2-2))

show(grid([[fig1],[fig2],[figM, figH]]))

La valeur de alpha est :  0.40803999999999996
à l'instant T = 20 :
L'erreur en norme 2 entre la solution calculée avec le shéma 1 et uf est :  0.09906345021355005
L'erreur en norme 2 entre la solution calculée avec le shéma 2 et uf est :  0.0026622509861568837


**Commentaire :**  
Pour dt = 0.004, on a $\alpha = 0.4 < 0.5$, les deux shémas convergent. On remarque que la masse est conservée avec le 1er shéma et pas par le 2ème, ce qui est conforme au résultat théorique. Néanmois, à T=20, l'erreur en norme 2 est de l'ordre de 0.1 pour le 1er shéma, et 0.003 pour le 2eme. Chacun des deux shémas présente alors des inconvénients et des avantages par rapport à l'autre.  
Par contre, pour dt = 0.005, $\alpha = 0.51 > 0.5$, aucun des deux shémas ne converge et on obtient des fonctions qui oscillent très fortement.

### Schéma implicite en temps

On considère le schéma "différences finies" implicite en temps donné par :

\begin{equation*}
\label{implicite}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = 0 \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1.
\end{equation*}

**Implémenter le schéma "différences finies" implicite en temps pour les deux conventions $(1)$ et $(2)$.**

**Commentaire :**  
Comme on aura dans cette partie besoin d'inverser des matrices tridiagonales, et pour pouvoir toujours utiliser le stockage tridiagonal, je vais reprendre les fonctions necéssaires pour celà que j'avait définies dans la PC6 (inversion en utilisant la décomposition LU)

In [225]:
# FONCTIONS REPRISES DE LA PC6 :

# decompLU_tridiag() prend en argument un stockage tridiagonal d'une matrice At
# et retourne le stockage tridiagonal de sa décompostion LU
def decompLU_tridiag(At) :
    b, a, c = At[0], At[1], At[2]
    N = len(b)
    alpha = np.zeros(N)
    gamma = np.zeros(N)
    alpha[0] = a[0]
    for i in range(1,N) :
        gamma[i-1] = c[i-1]/alpha[i-1]
        alpha[i] = a[i] - b[i]*gamma[i-1]
    Lt = gamma
    Ut = np.vstack((b, alpha))
    return Lt, Ut

# solve_L_descente() prend en paramètre Lt, correspondant au tridiagonal de L, et un vecteur f de même taille
# retourne la solution de L.y = f, calculée avec un algorithme de descente
def solve_L_descente(Lt,f) :
    y = np.zeros(len(f))
    y[0] = f[0]
    for i in range(1,len(f)) :
        y[i] = f[i] - Lt[i-1]*y[i-1]
    return y

# solve_U_remontee() prend en paramètre Ut, correspondant au tridiagonal de U, et un vecteur y
# retourne la solution de U.u = y, calculée avec un algorithme de remontée
def solve_U_remontee(Ut, y) :
    alpha, beta  = Ut[1], Ut[0]
    u = np.zeros(len(y))
    u[-1] = y[-1]/alpha[-1]
    for i in range(2,len(y)+1) :
        u[-i] = (y[-i] - beta[-i+1]*u[-i+1])/alpha[-i]
    return u

# solveLU_tridiag() prend en paramètre une matrice tridiagonale A, et un vecteur f
# retourne la solution de A.u = f, calculée avec un algorithme de remontée et descente
def solveLU_tridiag(A,f) :
    Lt, Ut = decompLU_tridiag(A)
    y = solve_L_descente(Lt,f)
    u = solve_U_remontee(Ut, y)
    return u

In [226]:
# La fonction resoudre_impl() prend en argument une matrice tridiagonale, un entier N (taille de t), un vecteur initial U0,
# elle renvoit le vecteur U^(N) calculé par A*U^(n+1) = U^(n), et les vecteurs de masse et d'energie correspondant à ce calcul
def resoudre_impl(A, N, U) :
    M = np.zeros(N+1) #masse
    H = np.zeros(N+1) #energie
    M[0] = masse(U)
    H[0] = dx*norme2(U)
    for n in range(1,N+1) :
        U = solveLU_tridiag(A,U)
        M[n] = masse(U)
        H[n] = dx*norme2(U)
    return U,M,H

**Pour chaque traitement des conditions aux bords $(1)$ et $(2)$, utiliser $\Delta t= 0.01$ et afficher les solutions obtenues pour $T=0$, $T=1$, $T=5$ et $T=20$  (sur un même graphique, avec différentes couleurs). Afficher également la masse $\sum_{j=0}^{J+1} U^{n+1}_j$ en fonction de $n$, et l'énergie $\left\Vert U^n \right\Vert_{L^2}$ en fonction de $n$. Commenter les différents résultats obtenus et discuter des avantages et des inconvénients des schémas explicites et implicites pour ce problème.**

In [227]:
# La matrice de la méthode explicite pour la 1ere convention au bords : -----------------------------------------------
def A1_impl(alpha) :
    return A1_expl(-alpha)

# La matrice de la méthode explicite pour la 2eme convention au bords : -----------------------------------------------
def A2_impl(alpha) :
    return A2_expl(-alpha)

In [228]:
# CHOSIR ICI LA VALEUR DE dt ET 3 VALEURS DE T NON NULLES (la courbe pour T=0 sera toujours tracée)
dt = 0.01
valeursT = [1,5,20]
#----------------------------------------------------------------------------------------------
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)

# Résolution et figure : ----------------------------------------------------------------------
# Calcul des matrices A1 et A2
A1 = A1_impl(alpha)
A2 = A2_impl(alpha)

col = {valeursT[0]:"green", valeursT[1]:"orange",valeursT[2]:"red" }

# 1er shéma
fig1 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 1 (méthode implicite)")
fig1.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T dans {1,5,20}, on calcule le vecteur U correspondant à l'instant T
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U1, M1, H1 = resoudre_impl(A1,N,U0)
    fig1.line(x, U1, color = col[T], legend = "T = "+str(T))

# 2ème shéma
fig2 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 2 (méthode implicite)")
fig2.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T, on calcule le vecteur U correspondant à l'instant T, et on le trace
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U2, M2, H2 = resoudre_impl(A2,N,U0)
    fig2.line(x, U2, color = col[T], legend = "T = "+str(T))

# évolution de la masse et de l'energie en fonction du temps pour les deux shémas
N = int(valeursT[-1]/dt)
t = np.array([n*dt for n in range(N+1)])

figM = figure(width = 490, height = 300, title = "Evolution de la masse en fonction du temps")
figM.line(t, M1, color = "red", legend = "pour le shéma 1")
figM.line(t, M2, legend = "pour le shéma 2")

figH = figure(width = 490, height = 300, title = "Evolution de l'energie en fonction du temps")
figH.line(t, H1, color = "red", legend = "pour le shéma 1")
figH.line(t, H2, legend = "pour le shéma 2")

print("à l'instant T = 20 :")
print("L'erreur en norme 2 entre la solution calculée avec le shéma 1 et uf est : ", norme2(U1-2))
print("L'erreur en norme 2 entre la solution calculée avec le shéma 2 et uf est : ", norme2(U2-2))

show(grid([[fig1],[fig2],[figM, figH]]))

La valeur de alpha est :  1.0200999999999998
à l'instant T = 20 :
L'erreur en norme 2 entre la solution calculée avec le shéma 1 et uf est :  0.09906553144740388
L'erreur en norme 2 entre la solution calculée avec le shéma 2 et uf est :  0.0027208471193991826


**Commentaire :**  
Les mêmes remarques faites en comparant les deux shémas pour la méthode explicite restent valables pour la méthode implicite.  
Toutefois, avec la méthode implicite, le problème de convergence ne se pose pas. On a démontré analytiquement que quelque soit la valeur de $\alpha$ on converge toujours vers la fonction $u_f$, et on le constate aussi expérimentalement : dans notre cas $\alpha > 1$.

## Étude d'une onde progressive

On considère l'équation de Nagumo :

\begin{equation*}
  \label{Nagumo_db}
  \left\{
  \begin{aligned}
  &\displaystyle\frac{\partial u}{\partial t}(t,x)-\nu\frac{\partial^2 u}{\partial x^2}(t,x) = g(u(t,x)), \quad &\forall~(t,x) \in\mathbb{R}_+^*\times\Omega,  \\
  &\frac{\partial u}{\partial x}(t,0) = 0 = \frac{\partial u}{\partial x}(t,L )\quad &\forall~t\in\mathbb{R}_+^*, \\
  &u(x,0) = u^0(x), \quad &\forall x\in\Omega,
  \end{aligned}
  \right.
\end{equation*}

où $g:\mathbb{R} \to \mathbb{R}$ est donnée par :

\begin{equation*}
\label{g}
g(u) = ku^2(1-u).
\end{equation*}

Pour les simulations, on prendra
\begin{equation*}
\nu=0.1,\quad L=100,\quad k=10,\quad \text{et}\quad u^0(x) =
\left\{
\begin{aligned}
& 1,\quad & x\leq \frac{L}{10} \\
& 0,\quad & x >\frac{L}{10}.
\end{aligned}
\right.
\end{equation*}

On utilisera les mêmes discrétisations spatiales et temporelles que dans l'exercice précédent, avec $J=1000$ ($\Delta x = \frac{L}{J+1}$).

### Schéma totalement explicite

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^n_{j-1}-2U^n_j+U^n_{j+1}}{(\Delta x)^2} = g(U^n_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

In [232]:
# La fonction resoudre_expl2() prend en argument une matrice tridiagonale, un entier N (taille de t), un vecteur initial U0,
# la valeur de dt, et la fonction g
# elle renvoit le vecteur U^(N) calculé par U^(n+1) = A*U^(n)
def resoudre_expl2(A, N, U, dt, g) :
    for n in range(1,N+1) :
        U = multiplier(A,U) + dt*g(U)
    return U

**Commentaire :**  
Je vais tracer en couleur grise, pour chaque instant T, la fonction $u^0(x-cT)$ correspondant à l'onde ayant l'allure de $u^0$ et se propageant à la vitesse $c$. (Il suffit de décommenter une ligne indiquée dans le code pour ne pas faire ce tracé)

In [233]:
nu = 0.1
L = 100
k = 10
c = np.sqrt(k*nu/2)
def u0(x) :
    return int(x<=L/10)
def g(u) :
    return k*u*u*(1-u)
def u0_propagée(x,t) :
    return np.array([u0(xj-c*t) for xj in x])

**Afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.025$ (sur un même graphique, avec différentes couleurs). Utiliser maintenant $\Delta t =0.045$ et afficher la solution obtenue à $T\approx 20$. Commenter *(on pourra calculer $\alpha = \frac{\nu\Delta t}{\Delta x}$)*.**

In [234]:
# CHOSIR ICI LA VALEUR DE dt ET 3 VALEURS DE T NON NULLES (la courbe pour T=0 sera toujours tracée)
dt = 0.025 ; valeursT = [20,40,60]
#dt = 0.045 ; valeursT = [19,20,21]

tracer_propagée = True
# DECOMMENTEZ LA LIGNE SUIVANTE SI VOUS NE VOULEZ PAS AFFICHER LES COURBES GRISES CORRESPONDANT A u0(x-cT), 
#tracer_propagée = False

#----------------------------------------------------------------------------------------------
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)

# Résolution et figure : ----------------------------------------------------------------------
A1 = A1_expl(alpha)
col = {valeursT[0]:"green", valeursT[1]:"orange", valeursT[2]:"red"}

fig1 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 1 (méthode explicite)")
fig1.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T dans {1,5,20}, on calcule le vecteur U correspondant à l'instant T
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    dx = 10
    U = resoudre_expl2(A1,N,U0,dt,g)
    if tracer_propagée :
        fig1.line(x, u0_propagée(x,T), color = "grey")
    fig1.line(x, U, color = col[T], legend = "T = "+str(T))

show(fig1)

La valeur de alpha est :  0.0025502500000000004


**Commentaire :**  
On remarque qu'on a bien un phénomène de propagation avec une légère déformation de l'onde. Mais on remarque aussi, en comparant avec les tracés de la fonction $u^0(x-cT)$, que la vitesse de propation est un peu inférieure à $c$.  
La valeur $T=20$ pour $dt = 0.045$ ne pose aucun problème.

### Schéma semi-implicite.

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = g(U^n_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

In [235]:
# La fonction resoudre_impl2() prend en argument une matrice tridiagonale, un entier N (taille de t), un vecteur initial U0,
# la valeur de dt, et la fonction g
# elle renvoit le vecteur U^(N) calculé par U^(n+1) = A*U^(n)
def resoudre_impl2(A, N, U, dt, g) :
    for n in range(1,N+1) :
        U = solveLU_tridiag(A,U + dt*g(U))
    return U

**afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.045$ (sur un même graphique, avec différentes couleurs). Utiliser maintenant $\Delta t =0.4$ et afficher la solution obtenue à $T\approx 20$. Commenter.**

In [236]:
# CHOSIR ICI LA VALEUR DE dt ET 3 VALEUTS DE T NON NULLES (T=0 sera toujours tracée)
dt = 0.045 ; valeursT = [20,40,60]
#dt = 0.4   ; valeursT = [19,20,21]

tracer_propagée = True
# DECOMMENTEZ LA LIGNE SUIVANTE SI VOUS NE VOULEZ PAS AFFICHER LES COURBES GRISES CORRESPONDANT A uf(x,T), 
#tracer_propagée = False

#----------------------------------------------------------------------------------------------
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)

# Résolution et figure : ----------------------------------------------------------------------
A1 = A1_impl(alpha)
col = {valeursT[0]:"green", valeursT[1]:"orange", valeursT[2]:"red"}

fig1 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec le shéma 1 (méthode implicite)")
fig1.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T dans {1,5,20}, on calcule le vecteur U correspondant à l'instant T
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U = resoudre_impl2(A1,N,U0,dt,g)
    if tracer_propagée :
        fig1.line(x, u0_propagée(x,T), color = "grey")
    fig1.line(x, U, color = col[T], legend = "T = "+str(T))

show(fig1)

La valeur de alpha est :  0.00459045


**Commentaire :**  
On a la même remarque que pour le shémà explicite concernant la vitesse de propagation (inférieure à $c$).  
Et cette fois, pour $dt = 0.4$ et $T\approx 20$, on obtient de mauvaises fonctions qui n'ont pas du tout l'allure souhaitée, notre algorithme n'est pas stable pour cette valeur de $dt$. Il faut donc imposer une certaine condition sur $dt$ pour garantir que $\alpha$ soit assez petit.

### Schéma totalement implicite (bonus).

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = g(U^{n+1}_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Bonus : Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

**Commentaire :**  
Commencons d'abord par implémenter la méthode de Newton pour calculer le zéro de la fonction $F$ définie dans l'énoncé.  
La formule de Newton proposée dans l'énoncé fait appel à la matrice jacobienne $DF$ de $F$. Comme $A_1^{impl}$ est une matrice tridiagonale, il est facile d'expliciter analytiquement les coordonnées de $F(V)$ pour un vecteur $V$ donné, le calcul des dérivées partielles se fait facilement aussi.  
J'ai défini la fonction *DF()* qui renvoie cette matrice jacobienne, et ensuite j'ai défini la fonction *zero_newton()* qui calcul le zéro de la fonction de la fonction F

In [237]:
##### F est une fonction de V mais qui dépend des paramètres A, U, dt et g
def F(V,A,U,dt,g) :
    return multiplier(A,V) - dt*g(V) - U

def dg(u) :
    return k*u*(2-3*u)

# DF retourne la matrice jacobienne de F en V
def DF(V,alpha,dg) :
    J = np.zeros((len(V),len(V)))
    for i in range(1,len(V)-1) :
        J[i][i-1] = J[i][i+1] = -alpha
        J[i][i] = (1+2*alpha) - dg(V[i])
    J[0][1] = -alpha
    J[0][0] = 1+alpha - dg(V[0])
    J[-1][-1] = 1 + alpha - dg(V[0])
    J[-1][-2] = -alpha
    return J

# zero_newton() prend en argument F et tous ses paramètres, ainsi que le nombre d'iteration max et la précision recherchée
def zero_newton(F,A,U,dt,g,dg, nitmax=100, eps=1e-6) :
    V = U[:]
    i = 0
    FV = F(V,A,U,dt,g)
    while(i<nitmax and norme2(FV)>1e-6) :
        V = V - np.linalg.solve(DF(V,alpha,dg),FV)
        FV = F(V,A,U,dt,g)
        i += 1
    return V

In [240]:
# TEST DE LA FONCTION zero_newton()
dt = 0.04
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)
A1 = A1_impl(alpha)


print("test de la fonction newton_zero : les normes successives de Fn(U_{n+1}) pour n<=10 sont")
ti = time.time()
for n in range(11) :
    V = zero_newton(F,A,U0,dt,g,dg)
    print(norme2(F(V,A1,U0,dt,g)))
    U0 = V
tf = time.time()
print("Le temps total du calcul est ",tf-ti,"secondes")

La valeur de alpha est :  0.0040804000000000005
test de la fonction newton_zero : les normes successives de Fn(U_{n+1}) pour n<=10 sont
9.090583488357999e-07
9.280570317669005e-07
9.645185246787657e-07
8.812544127533558e-07
9.302379869019378e-07
9.234042399074988e-07
0.007740008724640022
8.772283631416831e-07
0.008849614948350428
8.964639155616349e-07
0.006605791021020844
Le temps total du calcul est  2.149622678756714 secondes


**Commentaire :**  
En itérant la fonction zero_newton(), on remarque qu'elle nous donne de bonnes approximations des zéros de des fonctions $F^n$ : à chaque itération on a $F^n(U^{n+1}) \approx 0$. Toutefois notre fonction est lente, pour 10 itérations elle a besoin d'environ 2 secondes, son temps de calcul est en moyenne 0.2 seconde/itération.  
Rien que pour $T=20$ et dt=0.04 , elle aurait besoin de faire $T/dt = 500$ itérations, ce qui correspond à environ 1 minute et 40 secondes de calcul.

In [241]:
# La fonction resoudre_impl2() prend en argument une matrice tridiagonale, un entier N (taille de t), un vecteur initial U0,
# la valeur de dt, et la fonction g
# elle renvoit le vecteur U^(N) calculé par U^(n+1) = A*U^(n)
def resoudre_total_impl(A, N, U0, dt, g, dg, nitmax=100, eps=1e-6) :
    for n in range(1,N+1) :
        V = zero_newton(F,A,U0,dt,g,dg,nitmax,eps)
        U0 = V
    return V

**Bonus : Afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.4$ (sur un même graphique, avec différentes couleurs). Comparer avec les solutions obtenues, pour le même schéma, avec $\Delta t = 0.04$.**

In [242]:
# CHOSIR ICI LA VALEUR DE dt
dt = 0.4 ; 
#dt = 0.04
valeursT = [20,40,60]

tracer_propagée = True
# DECOMMENTEZ LA LIGNE SUIVANTE SI VOUS NE VOULEZ PAS AFFICHER LES COURBES GRISES CORRESPONDANT A uf(x,T), 
#tracer_propagée = False

#----------------------------------------------------------------------------------------------
dx = L/(J+1)
x = np.array([j*dx for j in range(J+2)])
U0 = np.array([u0(xj) for xj in x])
alpha = (nu*dt)/(dx*dx)
print("La valeur de alpha est : ",alpha)

# Résolution et figure : ----------------------------------------------------------------------
A1 = A1_impl(alpha)
col = {valeursT[0]:"green", valeursT[1]:"orange", valeursT[2]:"red"}

fig1 = figure(width = 850, height = 400, title = "Evolution de la fonction calculée avec la méthode totalement implicite")
fig1.line(x,U0, legend = "T = 0")
for T in valeursT : # Pour chaque valeur de T dans {1,5,20}, on calcule le vecteur U correspondant à l'instant T
    N = int(T/dt)
    t = np.array([n*dt for n in range(N+1)])
    U = resoudre_total_impl(A1,N,U0,dt,g, dg)
    if tracer_propagée :
        fig1.line(x, u0_propagée(x,T), color = "grey")
    fig1.line(x, U, color = col[T], legend = "T = "+str(T))

show(fig1)

La valeur de alpha est :  0.04080400000000001


**Commentaire :**  
Pour dt=0.4 et 0.04 on obtient un phénomène de propagation, sauf que la vitesse de propagation est trop grande pour dt=0.4 et trop petite pour dt=0.04